In [2]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karthikp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = pd.read_csv(r'C:\Users\karthikp\Downloads\Inceptors\Preprocessing files\BookPrediction_Data_Train.csv')
test = pd.read_csv(r'C:\Users\karthikp\Downloads\Inceptors\Preprocessing files\BookPrediction_Data_Test.csv')
submission = pd.read_csv(r'C:\Users\karthikp\Downloads\Inceptors\Preprocessing files\BookPrediction_Sample_Submission.csv')

In [4]:
train[:5]

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [5]:
test.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"


In [6]:
submission.head()

,Price
0,119
1,123
2,108
3,118
4,132


In [7]:
train.shape, test.shape

((6237, 9), (1560, 8))

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         6237 non-null   object 
 1   Author        6237 non-null   object 
 2   Edition       6237 non-null   object 
 3   Reviews       6237 non-null   object 
 4   Ratings       6237 non-null   object 
 5   Synopsis      6237 non-null   object 
 6   Genre         6237 non-null   object 
 7   BookCategory  6237 non-null   object 
 8   Price         6237 non-null   float64
dtypes: float64(1), object(8)
memory usage: 438.7+ KB


In [9]:
train.nunique()

Title           5568
Author          3679
Edition         3370
Reviews           36
Ratings          342
Synopsis        5549
Genre            345
BookCategory      11
Price           1614
dtype: int64

In [10]:
train.duplicated().sum()

0

In [11]:
test.duplicated().sum() # duplication will check for duplicated data with all the columns

8

In [12]:
test[test.duplicated()]

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1090,The Rustlers of the West Fork (Hopalong Cassidy),Louis L'Amour,"Mass Market Paperback,– 1 Apr 1992",5.0 out of 5 stars,2 customer reviews,"In this first of four classic frontier novels,...",Action & Adventure (Books),Action & Adventure
1095,The Princess Bride: S. Morgenstern's Classic T...,William Goldman,"Mass Market Paperback,– 26 Mar 2010",3.8 out of 5 stars,6 customer reviews,William Goldman's modern fantasy classic is a ...,Action & Adventure (Books),Action & Adventure
1103,Everybody Lies: The New York Times Bestseller,Seth Stephens-Davidowitz,"Paperback,– 18 Apr 2018",4.3 out of 5 stars,27 customer reviews,THE NEW YORK TIMES BESTSELLER\nAN ECONOMIST BO...,Political Structure & Processes (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1316,The Five Greatest Warriors (Jack West Series),Matthew Reilly,"Paperback,– 25 Nov 2010",4.4 out of 5 stars,12 customer reviews,The end is approaching ... Can Jack West unrav...,Action & Adventure (Books),Action & Adventure
1340,Alita: Battle Angel – The Official Movie Novel...,Pat Cadigan,"Paperback,– 14 Feb 2019",3.9 out of 5 stars,7 customer reviews,The ofﬁcial novelization to the highly anticip...,Action & Adventure (Books),Action & Adventure
1407,The Curious Incident of the Dog in the Night-t...,Mark Haddon,"Paperback,– 2 Aug 2012",4.0 out of 5 stars,96 customer reviews,A Children’s Bookshelf Selection: Each month o...,Action & Adventure (Books),Action & Adventure
1453,Storm and Silence: Volume 1 (Storm and Silence...,Robert Thier,"Paperback,– 19 Mar 2016",5.0 out of 5 stars,59 customer reviews,Freedom - that is what Lilly Linton wants most...,Action & Adventure (Books),Action & Adventure


In [13]:
test[test['Title']=='Flint']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
1171,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure
1253,Flint,Louis L'Amour,"Mass Market Paperback,– 3 Nov 1997",4.3 out of 5 stars,4 customer reviews,"He left the West at the age of seventeen, leav...",Action & Adventure (Books),Action & Adventure


In [14]:
train['source']='train'
test['source']='test'

In [15]:
data = pd.concat([train,test])

In [16]:
data.shape

(7797, 10)

In [17]:
data.isna().sum()

Title              0
Author             0
Edition            0
Reviews            0
Ratings            0
Synopsis           0
Genre              0
BookCategory       0
Price           1560
source             0
dtype: int64

In [18]:
data = data.reset_index().drop(columns = 'index')

In [19]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [20]:
data['Reviews'].apply(lambda x: print(x.split(' ')))

['4.0', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.9', 'out', 'of', '5', 'stars']
['3.5', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.8', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.9', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.9', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.1', 'out', 'of', '5', 'stars']
['3.1', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of',

['3.5', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.7', 'out', 'of', '5', 'stars']
['1.0', 'out', 'of', '5', 'stars']
['3.5', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of',

['3.4', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['3.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['2.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['1.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['3.7', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['3.1', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of',

['5.0', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.8', 'out', 'of', '5', 'stars']
['2.5', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['3.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.9', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of',

['4.1', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['2.8', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of',

...

['5.0', 'out', 'of', '5', 'stars']
['3.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['3.3', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['3.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.1', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of',

['4.0', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['3.8', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['3.4', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['3.5', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of',

['3.9', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.8', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['3.9', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.8', 'out', 'of', '5', 'stars']
['4.2', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['3.7', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of',

['5.0', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.6', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.4', 'out', 'of', '5', 'stars']
['4.7', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.1', 'out', 'of', '5', 'stars']
['4.3', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['4.5', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['5.0', 'out', 'of', '5', 'stars']
['3.0', 'out', 'of', '5', 'stars']
['3.6', 'out', 'of',

0       None
1       None
2       None
3       None
4       None
        ... 
7792    None
7793    None
7794    None
7795    None
7796    None
Name: Reviews, Length: 7797, dtype: object

In [21]:
data['Reviews'] = data['Reviews'].apply(lambda x: float(x.split(' ')[0]))

In [22]:
data['Reviews'].isna().sum()

0

In [23]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [24]:
data['Ratings'] = data['Ratings'].apply(lambda x: x.split(' ')[0])

In [25]:
data['Ratings'] = data['Ratings'].apply(lambda x: x.replace(",","")).astype(np.int)

C:\Users\karthikp\AppData\Local\Temp/ipykernel_14464/4081907056.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Ratings'] = data['Ratings'].apply(lambda x: x.replace(",","")).astype(np.int)


In [26]:
data['Ratings']

0        8
1       14
2        6
3       13
4        1
        ..
7792     4
7793    13
7794     5
7795     3
7796     2
Name: Ratings, Length: 7797, dtype: int32

In [27]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train


In [30]:
data['Edition']

0       Paperback,– 10 Mar 2016
1        Paperback,– 7 Nov 2012
2       Paperback,– 25 Feb 1982
3        Paperback,– 5 Oct 2017
4       Hardcover,– 10 Oct 2006
                 ...           
7792    Paperback,– 14 Apr 2011
7793     Paperback,– 8 May 2013
7794     Paperback,– 6 Sep 2011
7795    Paperback,– 22 Sep 2009
7796    Paperback,– 16 Sep 2016
Name: Edition, Length: 7797, dtype: object

In [31]:
data['Publication'] = data['Edition'].apply(lambda x: x.split(",")[0])
data['Publication_Date'] = data['Edition'].apply(lambda x: x.split(",")[1])

In [32]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,– 10 Mar 2016
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,– 7 Nov 2012
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,– 25 Feb 1982
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,– 5 Oct 2017
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,– 10 Oct 2006


In [33]:
data['Publication'].unique()

array(['Paperback', 'Hardcover', 'Mass Market Paperback', 'Sheet music',
       'Flexibound', 'Plastic Comb', 'Loose Leaf', 'Tankobon Softcover',
       'Perfect Paperback', 'Board book', 'Cards', 'Spiral-bound',
       '(Kannada)', 'Product Bundle', 'Library Binding', '(German)',
       'Leather Bound', '(French)', '(Spanish)', '(Chinese)'],
      dtype=object)

In [34]:
data['Publication'].value_counts()

Paperback                6458
Hardcover                1056
Mass Market Paperback     200
Sheet music                32
Flexibound                 19
Cards                       9
Spiral-bound                6
Loose Leaf                  3
Tankobon Softcover          3
Perfect Paperback           1
Board book                  1
Plastic Comb                1
(Kannada)                   1
Product Bundle              1
Library Binding             1
(German)                    1
Leather Bound               1
(French)                    1
(Spanish)                   1
(Chinese)                   1
Name: Publication, dtype: int64

In [35]:
major_publication = ['Paperback','Hardcover']

In [36]:
major_publication

['Paperback', 'Hardcover']

In [37]:
data['Publication'] = data['Publication'].apply(lambda x: x if x in major_publication else 'others')

In [41]:
data['Publication'].value_counts()

Paperback    6458
Hardcover    1056
others        283
Name: Publication, dtype: int64

In [42]:
data['Publication_Date'] = data['Publication_Date'].str[-4:]

In [43]:
data['Publication_Date'].unique() # Since it is bringing alpha values as well, we will be pulling this data directly from Edition column

array(['2016', '2012', '1982', '2017', '2006', '2009', 'port', '2015',
       '2013', '1999', '2002', '2011', '2018', '1991', '2014', 'Edge',
       ' set', '1989', '2000', '2005', '2019', '2008', '2004', '2010',
       'tion', 'dged', '2001', '1969', '1993', 'ated', '1992', '2003',
       '1997', '1995', '2007', ' DVD', '1987', 'rice', '1986', '1990',
       '1988', 'rint', '1976', '1994', '1996', '1998', '1974', '1983',
       '1971', '1985', 'book', '1978', '1984', '1980', '1964', '1979',
       'mile', 'back', '1960', '1970', 'Book', '1975', '1905', '1900',
       '1961', '1973', 'DPCM', '1981', '1942', '1972'], dtype=object)

In [44]:
data['Publication_Year']=data['Edition'].str[-4:]

In [45]:
data['Publication_Year'].unique()

array(['2016', '2012', '1982', '2017', '2006', '2009', '2018', '2015',
       '2013', '1999', '2002', '2011', '1991', '2014', '1989', '2000',
       '2005', '2019', '2008', '2004', '2010', '2007', '2001', '1969',
       '1993', '1992', '2003', '1996', 'port', '1997', '1995', 'NTSC',
       '1987', '1986', '1990', '1988', '1981', '1976', '1994', '1998',
       '1977', '1974', '1983', '1971', '1985', '1978', 'mile', ' set',
       'tion', '1964', '1984', '1980', 'dged', '1979', 'rint', '1960',
       '1970', '1975', '1905', '1900', 'book', '1961', '1925', '1973',
       'ated', '1942', '1972'], dtype=object)

In [46]:
data[data['Publication_Year']== 'port']

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year
169,Long Walk to Freedom: Illustrated Children's e...,Nelson Mandela,"Paperback,– Abridged, Import",5.0,2,Long Walk to Freedom by Nelson Mandela is the ...,Action & Adventure (Books),Action & Adventure,418.0,train,Paperback,dged,port
582,Fundamentals of Drawing Portraits: A Practical...,Barrington Barber,"Paperback,– Illustrated, Import",4.0,7,This book is essential for everyone who wants ...,"Handicrafts, Decorative Arts & Crafts (Books)","Arts, Film & Photography",415.0,train,Paperback,ated,port
972,"Cartooning, The Professional Step-by-Step Guid...","Ivan Hissey, Curtis Tappenden","Paperback,– Illustrated, Import",3.0,1,A comprehensive and practical guide to drawing...,"Handicrafts, Decorative Arts & Crafts (Books)",Comics & Mangas,445.0,train,Paperback,ated,port
1233,Amma Tell Me About Raksha Bandhan!,Bhakti Mathur,"Paperback,– Large Print, Import",5.0,1,A promise to love and to protect\nIs sealed by...,Action & Adventure (Books),Action & Adventure,277.0,train,Paperback,rint,port
1631,Swimming: Swimming Made Easy: Beginner and Exp...,Ace McCloud,"Paperback,– Large Print, Import",4.0,1,Are you tired of being a weak swimmer? Do less...,"Swimming, Snorkelling & Diving (Books)",Sports,709.0,train,Paperback,rint,port
2779,Madhubani Art: Indian Art Series,Bharti Dayal,"Paperback,– Illustrated, Import",5.0,4,Madhubani art's origin is believed to go back ...,Painting Books,"Arts, Film & Photography",805.0,train,Paperback,ated,port
3875,While the Light Lasts (The Agatha Christie Col...,Agatha Christie,"Hardcover,– Facsimile, Import",3.5,7,"The very last Agatha Christie book, including ...",Short Stories (Books),"Crime, Thriller & Mystery",896.0,train,Hardcover,mile,port
4036,Mountaineering: The Freedom of the Hills,The Mountaineers,"Paperback,– Illustrated, Import",5.0,1,"For nearly 60 years it's been revered as the ""...",Outdoor Survival Skills (Books),Sports,1826.0,train,Paperback,ated,port
4403,Indian Tibet Tibetan India: The Cultural Legac...,Peter Van Ham,"Hardcover,– Illustrated, Import",4.4,2,Inspired by the first cultural expedition into...,Travel & Holiday Guides (Books),"Arts, Film & Photography",1542.0,train,Hardcover,ated,port
6816,"The Spy Chronicles: RAW, ISI and the Illusion ...",A.S. Dulat,"Hardcover,– Illustrated, Import",3.5,132,Pointing to the horizon where the sea and sky ...,Politics (Books),Politics,NaN,test,Hardcover,ated,port


In [47]:
data['Publication_Year']=data['Publication_Year'].apply(lambda x: re.sub("[^0-9]+", 'NA',x))

In [48]:
data['Publication_Year'] = data['Publication_Year'].replace('NA',0)

In [49]:
data['Age_of_book'] = 2022 - data['Publication_Year'].astype(int)

In [50]:
data['Age_of_book'] = data['Age_of_book'].replace(2022, 0)

In [51]:
data['Age_of_book'].unique()

array([  6,  10,  40,   5,  16,  13,   4,   7,   9,  23,  20,  11,  31,
         8,  33,  22,  17,   3,  14,  18,  12,  15,  21,  53,  29,  30,
        19,  26,   0,  25,  27,  35,  36,  32,  34,  41,  46,  28,  24,
        45,  48,  39,  51,  37,  44,  58,  38,  42,  43,  62,  52,  47,
       117, 122,  61,  97,  49,  80,  50])

In [52]:
data.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price,source,Publication,Publication_Date,Publication_Year,Age_of_book
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0,8,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00,train,Paperback,2016,2016,6
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9,14,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93,train,Paperback,2012,2012,10
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8,6,"""During the time men live without a common Pow...",International Relations,Humour,299.00,train,Paperback,1982,1982,40
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1,13,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00,train,Paperback,2017,2017,5
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0,1,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62,train,Hardcover,2006,2006,16


In [54]:
data['Age_of_book'].mean()

10.067718353212774

In [55]:
data['Age_of_book'] = data['Age_of_book'].replace(0, 10)

In [56]:
# Trying to check the model score with the columns that have been pre-processed as of now,
# so that we can compare our score after further pre-proccessing

In [57]:
stop_words = set(stopwords.words('english'))

In [58]:
for i in data['Title'].head(10):
    print(i)
    string = ""
    i = re.sub("[^A-Za-z]"," ",i)
    for word in i.lower().split():
        if not word in stop_words:
            string += word + " "
    print('After removing')
    print(string)

The Prisoner's Gold (The Hunters 3)
After removing
prisoner gold hunters 
Guru Dutt: A Tragedy in Three Acts
After removing
guru dutt tragedy three acts 
Leviathan (Penguin Classics)
After removing
leviathan penguin classics 
A Pocket Full of Rye (Miss Marple)
After removing
pocket full rye miss marple 
LIFE 70 Years of Extraordinary Photography
After removing
life years extraordinary photography 
ChiRunning: A Revolutionary Approach to Effortless, Injury-Free Running
After removing
chirunning revolutionary approach effortless injury free running 
Death on the Nile (Poirot)
After removing
death nile poirot 
Yoga Your Home Practice Companion: A Complete Practice and Lifestyle Guide: Yoga Programmes, Meditation Exercises, and Nourishing Recipes (Sivananda Yoga Vedanta Centre)
After removing
yoga home practice companion complete practice lifestyle guide yoga programmes meditation exercises nourishing recipes sivananda yoga vedanta centre 
Karmayogi: A Biography of E. Sreedharan
After remo

In [50]:
data_filter = data[['Reviews','Ratings','Age_of_book','Publication','BookCategory','Price','source']]

In [51]:
data_filter_category = pd.get_dummies(data[['Publication','BookCategory']])

In [52]:
data_preprocessed = pd.concat([data_filter_category,data_filter.drop(columns=['Publication','BookCategory'])], axis = 1)

In [53]:
book_train_preprocessed = data_preprocessed[data_preprocessed['source']=='train']

In [54]:
book_test_preprocessed = data_preprocessed[data_preprocessed['source']=='test']

In [55]:
book_train_preprocessed.drop(columns = ['source'],inplace = True)

C:\Users\karthikp\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [56]:
book_test_preprocessed.drop(columns = ['source','Price'], inplace = True)

In [64]:
book_train_preprocessed.shape, book_test_preprocessed.shape

((6237, 18), (1560, 17))

In [65]:
book_test_preprocessed = book_test_preprocessed.reset_index().drop(columns = 'index')

In [66]:
book_train_preprocessed.to_csv('book_train_preprocessed.csv', index = False)
book_test_preprocessed.to_csv('book_test_preprocessed.csv', index = False)